# PTC analysis using the DM Software Stack

In this notebook we compare three ways of measuring gains:
* The formula used in the `welcome_to_PTC.ipynb`
* Directly from the slopes of PTC's
* From EO Test

### Imports

In [ ]:
%matplotlib inline

# system imports
from matplotlib import pylab as plt
import numpy as np
import os
from scipy import optimize

from astropy.table import Table

# LSST stack imports
from lsst.daf.persistence import Butler
import lsst.afw.display as afwDisplay
from lsst.ip.isr import IsrTask
import lsst.afw.math as afwMath


# Firefly client imports
from IPython.display import IFrame

In [ ]:
import matplotlib
matplotlib.rcParams['figure.dpi'] = 120

This notebook requires the package `obs_lsst`, which is not currently distributed in the LSP. If the following cell raises an exception, see the instructions in `welcome_to_FE55.ipynb`.

In [ ]:
import eups
assert eups.getSetupVersion("obs_lsst")

### Set up the config for the ISR task.  This essentially turns off all processing other than overscan and bias correction.

In [ ]:
isr_config = IsrTask.ConfigClass()

isr_config.doDark=False
isr_config.doFlat=False
isr_config.doFringe=False
isr_config.doDefect=False
isr_config.doAddDistortionModel=False
isr_config.doLinearize=False
isr_config.doSaturationInterpolation=False

### Construct the `IsrTask` with the above configuration

In [ ]:
isr = IsrTask(config=isr_config)

### Setup firefly to do image visualization

In [ ]:
my_channel = '{}_test_channel'.format(os.environ['USER'])
server = 'https://lsst-lspdev.ncsa.illinois.edu'


ff='{}/firefly/slate.html?__wsch={}'.format(server, my_channel)
IFrame(ff,800,600)

afwDisplay.setDefaultBackend('firefly')
afw_display = afwDisplay.getDisplay(frame=1, 
                                    name=my_channel)

In [ ]:
#For a chosen sensor, it is sometimes useful to know the name associated with the index
sensor_num = 0
sensor_names = ['S00', 'S01', 'S02', 'S10', 'S11', 'S12', 'S20', 'S21', 'S22']
sensor_name = sensor_names[sensor_num]

## Calculate information for all amps on a given sensor 

The function below will calculate means, variances, and gains for all flat image pairs in a specified repo

In [ ]:
def calculate_gains_full(rtm_path, sensor_num):
    
    '''
    @params
    -----
    rtm_path:   <str> Path to data repo for a certain raft
    sensor_num: <int> CCD number, must be in range(9)
    
    @returns
    -----
    results: <array>, shape=(n_visits / 2, 16)
                      elements: [('visit1', 'int'), ('visit2', 'int'), ('index', 'int'), 
                                 ('gain', 'f4'), ('exp_time', 'f4'), ('mean', 'f4'), 
                                 ('variance', 'f4')]
    
    '''
    # Initialize empty dictionaries for storing calculations
    gain, exp_time = {}, {}
    
    # Instantiate a butler and get individual exposures
    butler = Butler(rtm_path)
    visits = butler.queryMetadata('raw', ['visit'], dataId={'imageType': 'FLAT', 'testType': 'FLAT'}) ## what is 'testType' ??
    
    # Instantiate an empty array for storing all calculations
    nn = int(len(visits) / 2)
    results = np.empty((nn, 16), dtype=[('visit1', 'int'), ('visit2', 'int'), ('index', 'int'), ('gain', 'f4'), ('exp_time', 'f4'), ('mean', 'f4'), ('variance', 'f4')])
    
    # Loop over all visit pairs to calculate gains
    i = 0
    ## Assuming pairs are visits listed one after another
    for visit1, visit2 in zip(visits[:-1:2], visits[1::2]): # loop over pairs of images
        # Get ISR data for first image
        dId = {'visit': visit1, 'detector': sensor_num}
        raw1 = butler.get('raw', **dId)
        bias1 = butler.get('bias', **dId)
        time1 = raw1.getInfo().getVisitInfo().getExposureTime()

        # Get ISR data for second image
        dId = {'visit': visit2, 'detector': sensor_num}
        raw2 = butler.get('raw', **dId)
        bias2 = butler.get('bias', **dId)
        time2 = raw2.getInfo().getVisitInfo().getExposureTime()
        if abs(time1 - time2) > 0.01:
            "Mismatched exptimes"
            continue

        # run ISR on both images
        result1 = isr.run(raw1, bias=bias1)
        result2 = isr.run(raw2, bias=bias2)

        detector = result1.exposure.getDetector()
        
        #Initialize gain and exptime dictionaries with empty lists
        gain[visit1] = []
        exp_time[visit1] = []        
        
        # Loop over all the amps
        for hduidx in range(16):
            # Select amp
            amp = detector[hduidx]

            sub_im1 = result1.exposure.getMaskedImage()[amp.getBBox()]
            arr1 = sub_im1.getImage().getArray()
            sub_im2 = result2.exposure.getMaskedImage()[amp.getBBox()]
            arr2 = sub_im2.getImage().getArray()
            
            #Scale arr2 to have the same mean as arr1
            scaled_arr2 = arr2 / np.mean(arr2) * np.mean(arr1)
            #Get difference between the flats
            
            diff = arr1 - scaled_arr2

            # From RHL, 1/g = <(I1-I2)**2/(I1+I2)>
            diff_im = sub_im1.clone()
            diff_im -= sub_im2

            sum_im = sub_im1.clone()
            sum_im += sub_im2

            diff_im *= diff_im
            diff_im /= sum_im

            stats = afwMath.makeStatistics(diff_im, afwMath.MEDIAN | afwMath.MEAN)
            # Compute gain for this amp.
            gain[visit1].append(1/stats.getValue(afwMath.MEAN))
            exp_time[visit1].append(time1)
            
            #Output all results
            results[i,hduidx]['visit1'] = visit1
            results[i,hduidx]['visit2'] = visit2
            results[i,hduidx]['index'] = i
            results[i,hduidx]['gain'] = gain[visit1][hduidx]
            results[i,hduidx]['exp_time'] = exp_time[visit1][hduidx]
            results[i,hduidx]['mean'] = np.mean(arr1)
            results[i,hduidx]['variance'] = np.var(diff) / 2.
            
                    
        print(i+1, ' of %i' %nn)
        i += 1    
        
    return results

In [ ]:
# Set the target data repo -- use raft 7
rtm_path = '/project/bootcamp/repo_RTM-007/'

# Set the target sensor or loop over all sensors -- use sensor 0
ccd_data = []
#for ccd_num in range(1):
#    ccd_data.append(calculate_gains_full(rtm_path, ccd_num))
ccd_data.append(calculate_gains_full(rtm_path, 0))

The variable `ccd_data` is a list with 9 elements, one for each ccd. Each element is a (39 x 16) array of information about that ccd. For example, you can access the data a given amp on a given ccd via the following code:

In [ ]:
ccd = 0   #0-8
amp = 0   #0-15

print('Sensor: %i   Amp: %i' %(ccd, amp))
print('[Visit 1, Visit2, Index, Gain, Exp Time, Mean, Variance]')
print(ccd_data[ccd][:,amp])


Now for a given ccd, we can plot the PTC's and the gain distributions for each amp as follows:

In [ ]:
def plot_PTC(ccd_data, ccd_num):
    gain_calcs = []
    
    powerlaw = lambda x, amp, index: amp * (x**index)
    
    means, variances = [], []
    for amp in range(16):
        means.append([x[5] for x in ccd_data[ccd_num][:,amp]])
        variances.append([x[6] for x in ccd_data[ccd_num][:,amp]])
        
    fig, axs = plt.subplots(4,4, figsize=(15, 15))

    axs = axs.reshape((16,))

    amp_num = 0
    for ax in axs:
        ax.scatter(means[amp_num], variances[amp_num], s=4)
        ax.text(0.1, 0.9, 'Amp %i' %amp_num, transform = ax.transAxes)
        
        #fit a power law cropping off the last 8 points to avoid saturation
        logx = np.log10(means[amp_num][:-15])
        logy = np.log10(variances[amp_num][:-15])
        logyerr = 0.1 / np.asarray(variances[amp_num][:-15])
        fitfunc = lambda p, x: p[0] + p[1] * x
        errfunc = lambda p, x, y, err: (y - fitfunc(p, x)) / err
        pinit = [1.0, -1.0]
        out = optimize.leastsq(errfunc, pinit, args=(logx, logy, logyerr), full_output=1)
        pfinal = out[0]
        covar = out[1]
        index = pfinal[1]
        amp = 10.0**pfinal[0]
        ax.plot(means[amp_num], powerlaw(means[amp_num], amp, index), ls='--', color='black', lw=1)
        
        #get y-int for read noise
        yint = amp
        slope = index
        ax.text(0.1, 0.8, 'Intercept = %.2f' %yint, transform = ax.transAxes)
        #ax.text(0.1, 0.7, 'Gain %.2f' %(1. / slope), transform = ax.transAxes)
        
        #fit a quadratic leaving off the last three points
        quad_fit = np.polyfit(np.asarray(means[amp_num][:-3]), np.asarray(variances[amp_num][:-3]), 2)
        quad = np.poly1d(quad_fit)
        ax.plot(means[amp_num], quad(means[amp_num]), ls='--', color='red', lw=1)
        quad_int = quad(0.0)
        
        #fit a line leaving off the last three points
        line_fit = np.polyfit(np.asarray(means[amp_num][6:-15]), np.asarray(variances[amp_num][6:-15]), 1)
        line = np.poly1d(line_fit)
        ax.plot(means[amp_num], line(means[amp_num]), ls='--', color='magenta', lw=1)
        line_int = line(0.0)
        gain_calc = 1.0 / line_fit[0]
        
        gain_calcs.append(gain_calc)
        
        #print("Amp %i -- Intercepts: linear %4.4f   quadratic %4.4f    power-law %4.4f" %(amp_num, line_int, quad_int, yint))
        
        amp_num += 1

    #plt.xlabel("Mean = (M1 + M2) / 2")
    #plt.ylabel("Variance = STD(DIFF) ** 2 / 2")
    plt.show() 
    
    return gain_calcs


Determine the gains from the slopes

In [ ]:
ptc_curves = plot_PTC(ccd_data, 0)

In [ ]:
print(ptc_curves)

Determine gains from the formula in the notebook, taking the median of all visits

In [ ]:
def plot_gains(ccd_data, ccd_num):
    gains = []
    meds = []
    
    for amp in range(16):
        gains.append([x[3] for x in ccd_data[ccd_num][:,amp]])
    
    fig, axs = plt.subplots(4,4, figsize=(15, 15))

    axs = axs.reshape((16,))

    amp_num = 0
    for ax in axs:
        med = np.median(gains[amp_num])
        meds.append(med)
        bins = np.linspace(min(gains[amp_num]), max(gains[amp_num]), 50)
        ax.hist(gains[amp_num], bins=bins, alpha=0.5, label='Amp %i' %amp_num)
        ax.text(0.6, 0.6, "Median: %.3f" %med, transform=ax.transAxes)
        ax.set_xlabel("Gain")
        ax.legend()    
        amp_num += 1

    plt.show()
    
    return meds

In [ ]:
rhl_gains = plot_gains(ccd_data, 0)

In [ ]:
print(rhl_gains)

Now get gains from EOTEST

In [ ]:
eo_test_gains = list(Table.read('E2V-CCD250-260_eotest_results.fits')['PTC_GAIN'])
print(eo_test_gains)

Plot all three methods...

In [ ]:
plt.figure()
plt.scatter(range(1,17), rhl_gains, label='Welcome to PTC Method')
plt.scatter(range(1,17), ptc_curves, label='PTC Slope')
plt.scatter(range(1,17), eo_test_gains, label='EOTEST')
plt.xlabel("Amp")
plt.ylabel("Gain")
plt.legend()
plt.show()

We are curious why the `Welcome to PTC` method and the `PTC Slope` method appear to be separated by a constant factor.

The remainder of this notebook is just scratch work

### Plot gain versus exposure time

In [ ]:
def plot_gains_versus_exposure(ccd_data, ccd_num):
    gain_means, exps_means, gain_stds, exps_stds = [], [], [], []

    for amp in range(16):
        gain_means.append(np.mean([x[3] for x in ccd_data[ccd_num][:,amp]]))
        exps_means.append(np.mean([x[4] for x in ccd_data[ccd_num][:,amp]]))
        gain_stds.append(np.std([x[3] for x in ccd_data[ccd_num][:,amp]]))
        exps_stds.append(np.std([x[4] for x in ccd_data[ccd_num][:,amp]]))
        
    #determine when sensor starts to get saturated
    thresh_noise_to_sig = 0.1
    noise_to_sig = np.asarray(gain_stds) / np.asarray(gain_means)
    sat_start = 0
    while sat_start < len(noise_to_sig):
        if noise_to_sig[sat_start] > thresh_noise_to_sig:
            break
        sat_start += 1

    plt.figure()
    amp_num = 0
    while amp_num < 16:
        plt.plot(ccd_3_gains[amp_num]['exp_time'], ccd_3_gains[amp_num]['gain'], label='Amp %i' %amp_num, lw=0.3)
        amp_num += 1

    plt.axvline(x=exps_means[sat_start], color='blue', ls='dashed', lw=2, label='Spread begins')

    plt.errorbar(exps_means, gain_means, xerr=exps_stds, yerr=gain_stds, 
                 fmt='s', markersize=2, color='black', capsize=2, label='Mean $\pm$ STD')

    plt.legend(ncol=2)
    plt.xlabel("Exposure Time (sec)")
    plt.ylabel("Gain (electrons / ADU)")
    plt.title("Sensor %i" %ccd_num)
    plt.show()

In [ ]:
plot_gains_versus_exposure(ccd_data, 0)

In [ ]:
#Plot log gain versus log exposure time for all amps
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(11,5))

amp_num = 0
while amp_num < 16:
    ax1.loglog(ccd_3_gains[amp_num]['exp_time'], ccd_3_gains[amp_num]['gain'], label='Amp %i' %amp_num, lw=0.3)
    ax2.plot(ccd_3_gains[amp_num]['exp_time'], ccd_3_gains[amp_num]['gain'], label='Amp %i' %amp_num, lw=0.3)
    amp_num += 1

ax1.set_title("Sensor 3")
ax1.set_ylabel("Gain (electrons / ADU)")
ax1.set_xlabel("Exposure Time (sec)")
ax1.legend(ncol=2)
ax2.set_title("Sensor 3")
ax2.set_ylabel("Gain (electrons / ADU)")
ax2.set_xlabel("Exposure Time (sec)")
ax2.legend(ncol=2)
plt.show()


In [ ]:
#Find exposure time at which gain starts to go wild
for amp_num in range(16):
    gains = sorted(ccd_3_gains[amp_num]['gain'])

## End test area

# Compare

In [ ]:
eo_test_gains = list(Table.read('E2V-CCD250-260_eotest_results.fits')['PTC_GAIN'])
print(eo_test_gains)

In [ ]:
BOOTCAMP_REPO_DIR= '/project/bootcamp/repo_RTM-007/'
butler = Butler(BOOTCAMP_REPO_DIR)
## butler.getKeys('raw')
## help(butler)
visits = butler.queryMetadata('raw', ['visit'], dataId={'imageType': 'FLAT', 'testType': 'FLAT'})
gain = {}
exp_time = {}

i = 1
for visit1, visit2 in zip(visits[:-1:2], visits[1::2]): # loop over pairs of images
    # Get ISR data for first image
    dId = {'visit': visit1, 'detector': 2}
    raw1 = butler.get('raw', **dId)
    bias1 = butler.get('bias', **dId)
    time1 = raw1.getInfo().getVisitInfo().getExposureTime()
    
    # Get ISR data for second image
    dId = {'visit': visit2, 'detector': 2}
    raw2 = butler.get('raw', **dId)
    bias2 = butler.get('bias', **dId)
    time2 = raw2.getInfo().getVisitInfo().getExposureTime()
    if abs(time1 - time2) > 0.01:
        "Mismatched exptimes"
        continue
    
    # run ISR on both images
    result1 = isr.run(raw1, bias=bias1)
    result2 = isr.run(raw2, bias=bias2)
    
    detector = result1.exposure.getDetector()
    amp = detector[3]

    sub_im1 = result1.exposure.getMaskedImage()[amp.getBBox()]
    #arr1 = sub_im1.getImage().getArray()
    sub_im2 = result2.exposure.getMaskedImage()[amp.getBBox()]
    #arr2 = sub_im2.getImage().getArray()
    
    # From RHL, 1/g = <(I1-I2)**2/(I1+I2)>
    diff_im = sub_im1.clone()
    diff_im -= sub_im2
    
    sum_im = sub_im1.clone()
    sum_im += sub_im2
    
    diff_im *= diff_im
    diff_im /= sum_im
    
    stats = afwMath.makeStatistics(diff_im, afwMath.MEDIAN | afwMath.MEAN)
    # Compute gain for this amp.
    gain[visit1] = 1/stats.getValue(afwMath.MEAN)
    exp_time[visit1] = time1
    print("visit %i,%i -- %i of %i -- gain=%f, exposure time(s)=%f"%(visit1, visit2, i, len(visits)/2, gain[visit1], exp_time[visit1], ))
    i += 1

In [ ]:
afw_display.mtv(sub_im1) # display an example image

In [ ]:
visit_keys = exp_time.keys()
x = [exp_time[visit] for visit in visit_keys]
y = [gain[visit] for visit in visit_keys]

In [ ]:
plt.scatter(x, y)
plt.ylim(0, 1)